# Food Bank Problem

In [3]:
import sys
import numpy as np
import plotly.express as px
import pandas as pd
import scipy.optimize as optimization
from food_bank_functions import *

## OPT - Waterfilling

In [39]:
# Auxilary function that defines the optimal policy as a function of the threshold, budget, and demand.
def policy(threshold, budget, demand):
    if threshold <= min(budget, demand):
        return threshold
    else:
        return min(budget, demand)

In [80]:
# Calculates the Bayes Optimal solution to the problem by solving the dynamic programming
# Note that this method was specified for the demand distribution where it is 1 with probability 1/2
# and 2 with probability 1/2

def bayes_opt(n, budget, b_grid, grid_size):
    
    # Stores the optimal thresholds for each stage in the algorithm
    opt_policy = np.zeros((n,len(b_grid)))
    
    # Stores the expected value function - where the expectation is taken with respect to the demand distribution
    v_fn = np.zeros((n, len(b_grid)))
    
    # Solves the Bellman recursion by first looping backwards through time
    for t in np.arange(n-1,-1,-1):
        
        # Then loops over each discretized budget
        for b in range(len(b_grid)):
            
            current_budget = b*grid_size
            
            # the optimal policy is to give out the rest
            if t == n-1:
                opt_policy[t,b] = float('inf')
                v_fn[t,b] = (1/2)*np.log(policy(opt_policy[t,b],current_budget,1)/1) + (1/2)*np.log(policy(opt_policy[t,b],current_budget,2)/2)
            
            
            else:
                
                # log(x) + V(t+1, b-x)
                q_vals = np.log(b_grid[0:(b+1)]) + np.flip(v_fn[t+1,0:(b+1)])  
                opt_policy[t,b] = np.argmax(q_vals)*grid_size
                
                new_budget_one = int(np.floor((current_budget - policy(opt_policy[t,b],current_budget,1))/grid_size))
                new_budget_two = int(np.floor((current_budget - policy(opt_policy[t,b],current_budget,2))/grid_size))
                
                v_fn[t,b] = (1/2)*(np.log(policy(opt_policy[t,b],current_budget,1)/1)+v_fn[t+1, new_budget_one]) \
                        + (1/2)*(np.log(policy(opt_policy[t,b],current_budget,2)/2)+v_fn[t+1, new_budget_two])
                # first calculate the threshold
    
    
    return opt_policy, v_fn

In [81]:
budget = 15
n = 10
grid_size = .1
b_grid = np.arange(0, budget+grid_size, grid_size)

In [77]:
v_fn[0,0:5]

array([-inf, -inf, -inf, -inf, -inf])

In [82]:
opt_policy, v_fn = bayes_opt(n, budget, b_grid, grid_size)

C:\Users\seanr\anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning:

divide by zero encountered in log

C:\Users\seanr\anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning:

divide by zero encountered in log

C:\Users\seanr\anaconda3\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning:

divide by zero encountered in log

C:\Users\seanr\anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning:

divide by zero encountered in log



In [85]:
print(v_fn[0,:])

[        -inf         -inf         -inf         -inf         -inf
         -inf         -inf         -inf         -inf         -inf
 -26.49158683 -25.79843965 -25.10529247 -24.41214529 -23.71899811
 -24.41214529 -22.33270375 -21.63955657 -20.94640939 -20.25326221
 -19.56011503 -19.15464992 -18.74918481 -18.3437197  -17.93825459
 -17.93825459 -17.53278949 -16.72185927 -16.4341772  -16.02871209
 -15.62324698 -15.62324698 -15.04788284 -14.64241773 -14.35473566
 -14.06705358 -14.06705358 -13.49168944 -13.20400737 -12.91632529
 -12.62864322 -12.40549967 -12.18235612 -11.95921257 -11.73606902
 -11.73606902 -11.28978191 -11.28978191 -10.84349481 -10.62035126
 -10.39720771 -10.21488615 -10.03256459  -9.85024304  -9.66792148
  -9.48559992  -9.30327837  -9.30327837  -8.96680613  -8.78448457
  -8.63033389  -8.47618321  -8.78448457  -8.16033027  -7.97800871
  -7.82385803  -7.66970735  -7.66970735  -7.36140599  -7.2278746
  -7.07372392  -6.91957324  -7.2278746   -6.63189116  -6.49835977
  -6.364828

In [15]:
# Bayes optimal version of water filling

def waterfilling_bayes_opt(town_demands,budget):
    n = np.size(town_demands)
    allocations = np.zeros(n)
    bundle_remaining = budget
    # Computes a random grid-size to use for the computation
    grid_size = budget / (5*n)
    # Computes othe Q_vals for the Bayes Optimal solution.
    Q_vals = bayes_opt(n, budget, grid_size)
    
    # For each town
    for i in range(n):
        # Compute the index of the remaining budget
        index = int(np.floor(bundle_remaining / grid_size))
        # Take the action that maximizes the Q function of their demands
        allocations[i] = np.argmax(Q_vals[index, 1:(index+1),(town_demands[i]-1).astype(int), i])*grid_size
        # Reduces the remaining budget
        bundle_remaining = bundle_remaining - allocations[i]
    return allocations

In [19]:
def make_demands_discrete_distribution(num_towns):
    demands = np.zeros(num_towns)
    expected_demands = np.zeros(num_towns)
    for i in range(num_towns):
        demands[i] = np.random.binomial(1,.5) + 1
        expected_demands[i] = 1.5
    return demands, expected_demands

In [20]:
num_iterations = 1
num_towns_range = 100
demands_max = 20
max_n = 1000
fix_num_towns = 10
max_b = 200


In [21]:
np.logspace(1,3).astype(int)

array([  10,   10,   12,   13,   14,   15,   17,   19,   21,   23,   25,
         28,   30,   33,   37,   40,   44,   49,   54,   59,   65,   71,
         79,   86,   95,  104,  115,  126,  138,  152,  167,  184,  202,
        222,  244,  268,  294,  323,  355,  390,  429,  471,  517,  568,
        625,  686,  754,  828,  910, 1000])

### Regret for Discrete Distribution of Demands

In [23]:
data_dict_1 = {'NumTowns':[],'Alg_1':[],'Alg_3':[], 'Alg_2':[]}

for n in np.logspace(1,2,10).astype(int):
    n+=1
    for i in range(num_iterations):
        data_dict_1['NumTowns'].append(n)
        town_demands, town_expected_demands = make_demands_discrete_distribution(n)
        budget = np.sum(town_expected_demands)

        opt = objective(town_demands, waterfilling(town_demands,budget))
        for j in range(3):
            if j==0:
                data_dict_1['Alg_1'].append(opt - objective(town_demands, waterfilling(town_expected_demands,budget)))
            if j==1:
                 data_dict_1['Alg_2'].append(opt - objective(town_demands, waterfilling_bayes_opt(town_demands,budget)))
            if j==2:
                data_dict_1['Alg_3'].append(opt - objective(town_demands, waterfilling_dynamic(town_expected_demands,town_demands,budget)))
df_uniform = pd.DataFrame(data_dict_1).melt(id_vars="NumTowns")
fig = px.scatter(df_uniform, x="NumTowns", y="value", color='variable')
fig.show()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:5: RuntimeWarning:

divide by zero encountered in log

